## 이 문서를 수정할 당신에게...
#### 현재 상황은 아래와 같습니다.
1. 현재 Assistant와 Agent에 대해 evaluation set과 few shot examples를 위한 우리 챗봇내에서 일어날만한 대화 데이터셋을 제작해야합니다.

#### 당신의 목표는 아래와 같습니다.
1. 최대한 우리 챗봇에서 일어날만한 대화 데이터셋 100개 제작을 목표로 합니다.

#### 현재 제가 생각한 방식입니다.
1. GPT4 API를 사용한 데이터셋 제작
- GPT 3.5는 한국어에 대한 성능과 대화셋 제작에 대한 성능이 부족하기 때문에 GPT4 API를 활용합니다.
- 아래 논문을 확인하여 프롬프트를 작성하고, 데이터셋을 수집합니다.
[Building a Role Specified Open-Domain Dialogue System Leveraging Large-Scale Language Models 논문](https://arxiv.org/pdf/2205.00176.pdf)
- 선택사항이지만 확장성을 위해 Azure OpenAI API보다 Langchain을 활용해 개발하는 것을 추천합니다.

#### 예상되는 문제
1. 데이터 수집에서 바닐라 GPT-4와 실제로 우리가 사용하는 챗봇은 다릅니다. 또한 실제 유저가 하는 말과 GPT-4가 하는 말은 다릅니다. 이로인해 실제 배포 환경과 다른 대화 셋입니다. 이 때문에 일단은 적은 수의 대화셋만 수집합니다. 
2. 우리의 챗봇은 실제 DB에 있는 데이터를 사용하는 등의 외부의 데이터를 사용하여 말을 생성합니다. 이 또한 실제 환경과 다른 데이터셋이 생성될 것입니다.

#### Future works
1. 실제 데이터와의 괴리를 줄이기 위해 우리의 챗봇과 사람이 대화할 수 있는 UI를 제작합니다.
2. 유저 프롬프트 등을 작성하여 실제 유저와 최대한 유사한 유저 언어모델을 돌릴 수 있도록 합니다. 즉 데이터셋 생성에서 유저 언어모델과 챗봇 언어모델 2가지의 모델이 돌아가면서 대화할 수 있도록 합니다.
3. 유저에 다양한 페르소나를 부여하여 좀 더 다양성있는 데이터를 수집하도록 합니다.
4. 언어모델의 문제 중에 hallucination(거짓말)과 성별,종교에 편향되거나 폭력적인 말을 하는 문제가 있습니다. 그리고 이를 유도하는 유저의 말이 있습니다. 이런 것들만을 위한 대화 데이터셋도 구성될 필요가 있습니다.(최근 네이버에서 이런 문제를 유발하는 한국어 데이터셋을 공개하기도 하고, 이에 관련한 영어 데이터셋도 많이 존재합니다.)

#### 참고 사항
아래는 위의 작업을 수행하기 위한 좋은 자료들입니다. 위의 문제해결 전에 아래 자료들을 먼저 학습하는 것을 매우 강력하게 추천합니다.
- [프롬프트 엔지니어링 강의 2시간](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/)
- [프롬프트 엔지니어링 가이드 documentation](https://www.promptingguide.ai/techniques)

아래 논문은 직접적이진 않지만 비싼 사람 데이터대신 언어모델을 사용하여 데이터를 수집하는 방식에 근간이 된 논문입니다.
- [Self-Instruct: Aligning Language Models with Self-Generated Instructions](https://arxiv.org/pdf/2212.10560.pdf)

----------------------------------------------------------------

### API 키 불러오기

In [1]:
import os
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('./secrets.ini')

['./secrets.ini']

In [3]:
openai_api_key = config['OPENAI']['OPENAI_API_KEY']
os.environ.update({'OPENAI_API_KEY': openai_api_key})

### 바닐라 GPT-4 생성

few-shot 기반으로 [대화 예시 json 파일](./templates/examples/dialog_generate_prompt_examples.json) 내부에 있는 대화 예시 중 일부를 골라서 프롬프트에 삽입한다.

생성을 위한 프롬프트는 [대화 생성 프롬프트 템플릿 파일](./templates/dialog_generate_prompt_template.txt)에 있다.

In [38]:
from __future__ import annotations

import re
from typing import List, Union, Dict, Optional

from ruamel.yaml import YAML
import numpy as np
import pandas as pd
from langchain import SerpAPIWrapper, LLMChain
from langchain.agents import Tool, AgentType, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.document_loaders import DataFrameLoader, SeleniumURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.prompts import PromptTemplate, StringPromptTemplate, load_prompt, BaseChatPromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.schema import AgentAction, AgentFinish, HumanMessage, BaseOutputParser
from langchain.vectorstores import DocArrayInMemorySearch, Chroma

Example Selector를 통해 미리 수집된 대화 데이터셋에서 랜덤한 대화를 하나 뽑습니다. 그리고 이를 프롬프트에 삽입합니다.

In [39]:
yaml = YAML()

class CustomExampleSelector(BaseExampleSelector):
    def __init__(self, file_path:str):
        with open(file_path, 'r', encoding='utf-8') as f:
            data = yaml.load(f)
        self.examples = [item for item in data]   

    def add_example(self, example: Dict[str, str]) -> None:
        """Add new example to store for a key."""
        self.examples.append(example)

    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """Select which examples to use based on the inputs."""
        return np.random.choice(self.examples, size=1, replace=False)

In [40]:
example_selector = CustomExampleSelector('./templates/examples/dialog_generate_prompt_examples.yaml')

In [42]:
example_selector.select_examples({})

array([{'user persona': '유저는 와인을 접한지 얼마 되지 않아 와인에 대한 지식이 없습니다. 그러나 와인을 탐구하고 새로운 것을 배우는 것에 흥미를 느낍니다. 그는 신선한 과일 향, 특히 체리 향이 나는 와인을 좋아합니다. 그의 예산은 7만원 미만이며 실용적인 구매 방식을 취합니다. 또한, 와인을 고를 때 다양한 옵션을 고려하며 여러 선택지 중에서 결정을 내리는 것을 좋아합니다.', "user's goal": '유저는 체리 향이 나고 너무 세지 않은 와인을 찾고 구매하려고 합니다. 유저의 예산은 7만원 이하이며, 여러 가지 제안을 바탕으로 정보에 입각한 결정을 내릴 수 있도록 원합니다.', 'conversation': "유저: 체리향 나는 와인 찾고 있는데 추천 좀 해줄 수 있을까요?\n어시스턴트: 네, 화이트 와인과 레드 와인 중 어떤 것을 선호하시나요?\n유저: 흠, 그런거까지는 잘 몰라요. \n어시스턴트: 그럼 와인의 단맛에 대한 선호도를 알려주실 수 있을까요? 달콤하거나 건조한 것 중 어느 것을 선호하시나요?\n유저: 아무래도 달달한 걸 좋아하는 편인 거 같아요.\n어시스턴트: 마지막으로 예산은 어느 정도 생각하고 계신가요?\n유저: 7만원 이하로 생각하고 있어요.\n어시스턴트: 그럼 '투 티크 대모자' 와인을 추천드립니다. 이 와인은 세미 스위트화이트 와인으로 체리 향이 나며 가격이 6만원대입니다. 어떻게 생각하시나요?\n유저: 오 좋아. 아무래도 안정적인 선택이 필요해서 다른 와인도 좀 더 추천해 주시겠어요?\n어시스턴트: 네, '빌라 마리아 셀라 스타피노 그리지오'와 '조르디에르 블랑 드 블랑'도 추천드릴 수 있습니다. 두 와인 모두 체리향이 나며 세미 스위트화이트 와인입니다. 가격대도 7만원 미만이어서 예산에 맞습니다. 이 중에서 마음에 드는 와인이 있나요?\n"}],
      dtype=object)

아래는 프롬프트 템플릿 입니다. prefix는 예시 대화가 들어가기 전에 나오는 내용이고, suffix는 예시 대화가 나온 후에 나오는 내용입니다.

In [43]:
prefix_prompt_template = """
여러분의 역할은 와인을 추천하는 어시스턴트와 유저 사이의 대화를 생성하는 것입니다.

아래 단계에 따라 문제를 해결하세요.
1단계. 유저의 페르소나를 생성합니다.
2단계. 유저가 달성하고자 하는 목표를 생성합니다.
3단계. 유저의 페르소나와 유저의 목표를 기반으로 대화를 생성합니다.

아래는 예시입니다.
"""

In [44]:
suffix_prompt_template = """
이제 정해진 순서대로 유저와 어시스턴트 간의 대화를 생성합니다.
"""

In [45]:
example_prompt = PromptTemplate(
    input_variables=["user persona", "user's goal", "conversation"], 
    template="유저 페르소나:{user persona}\n유저의 목표:{user's goal}\n대화:\n{conversation}")

In [46]:
prompt = FewShotPromptTemplate(
    prefix=prefix_prompt_template,
    example_selector=example_selector, 
    example_prompt=example_prompt,
    suffix=suffix_prompt_template, 
    input_variables=[]
)

아래는 llm이 출력한 대화를 일정한 파이썬 딕셔너리 형식으로 변경해줍니다. 아래는 예시입니다.
```python
{'user persona': '유저는 와인을 접한지 얼마 되지 않아 와인에 대한 지식이 없습니다. 그러나 와인을 탐구하고 새로운 것을 배우는 것에 흥미를 느낍니다. 그는 신선한 과일 향, 특히 체리 향이 나는 와인을 좋아합니다. 그의 예산은 7만원 미만이며 실용적인 구매 방식을 취합니다. 또한, 와인을 고를 때 다양한 옵션을 고려하며 여러 선택지 중에서 결정을 내리는 것을 좋아합니다.',
"user's goal": '유저는 체리 향이 나고 너무 세지 않은 와인을 찾고 구매하려고 합니다. 유저의 예산은 7만원 이하이며, 여러 가지 제안을 바탕으로 정보에 입각한 결정을 내릴 수 있도록 원합니다.',
'conversation': '''
    유저: 체리향 나는 와인 찾고 있는데 추천 좀 해줄 수 있을까요?
    어시스턴트: 네, 화이트 와인과 레드 와인 중 어떤 것을 선호하시나요?
    유저: 흠, 그런거까지는 잘 몰라요. 
    어시스턴트: 그럼 와인의 단맛에 대한 선호도를 알려주실 수 있을까요? 달콤하거나 건조한 것 중 어느 것을 선호하시나요?
    유저: 아무래도 달달한 걸 좋아하는 편인 거 같아요.
    어시스턴트: 마지막으로 예산은 어느 정도 생각하고 계신가요?
    유저: 7만원 이하로 생각하고 있어요.
    어시스턴트: 그럼 '투 티크 대모자' 와인을 추천드립니다. 이 와인은 세미 스위트화이트 와인으로 체리 향이 나며 가격이 6만원대입니다. 어떻게 생각하시나요?
    유저: 오 좋아. 아무래도 안정적인 선택이 필요해서 다른 와인도 좀 더 추천해 주시겠어요?
    어시스턴트: 네, '빌라 마리아 셀라 스타피노 그리지오'와 '조르디에르 블랑 드 블랑'도 추천드릴 수 있습니다. 두 와인 모두 체리향이 나며 세미 스위트화이트 와인입니다. 가격대도 7만원 미만이어서 예산에 맞습니다. 이 중에서 마음에 드는 와인이 있나요?
'''
```

In [58]:
class CustomRegexParser(BaseOutputParser):
    """Class to parse the output into a dictionary."""

    regex: str
    output_keys: List[str]
    default_output_key: Optional[str] = None

    @property
    def _type(self) -> str:
        """Return the type key."""
        return "regex_parser"

    def parse(self, text: str) -> Dict[str, str]:
        """Parse the output of an LLM call."""
        match = re.search(self.regex, text, re.DOTALL)
        if match:
            return {key: match.group(i + 1) for i, key in enumerate(self.output_keys)}
        else:
            if self.default_output_key is None:
                raise ValueError(f"Could not parse output: {text}")
            else:
                return {
                    key: text if key == self.default_output_key else ""
                    for key in self.output_keys
                }

In [86]:
parser = CustomRegexParser(regex=r'(?:유저 페르소나: )(.*)\n(?:유저의 목표: )(.*)\n(?:대화:)(.*)', output_keys = ['user persona', "user's goal", 'conversation'])

이제부터 실제로 대화를 생성해봅시다.

In [87]:
llm = ChatOpenAI(model='gpt-4', temperature=1.0)

In [88]:
dialog_generate_chain = LLMChain(
    llm=llm,
    prompt=prompt, 
    verbose=True, # 과정을 출력할지
    )

In [52]:
result = dialog_generate_chain.run({})



> Entering new  chain...
Prompt after formatting:

여러분의 역할은 와인을 추천하는 어시스턴트와 유저 사이의 대화를 생성하는 것입니다.

아래 단계에 따라 문제를 해결하세요.
1단계. 유저의 페르소나를 생성합니다.
2단계. 유저가 달성하고자 하는 목표를 생성합니다.
3단계. 유저의 페르소나와 유저의 목표를 기반으로 대화를 생성합니다.

아래는 예시입니다.


유저 페르소나:유저는 와인을 접한지 얼마 되지 않아 와인에 대한 지식이 없습니다. 그러나 와인을 탐구하고 새로운 것을 배우는 것에 흥미를 느낍니다. 그는 신선한 과일 향, 특히 체리 향이 나는 와인을 좋아합니다. 그의 예산은 7만원 미만이며 실용적인 구매 방식을 취합니다. 또한, 와인을 고를 때 다양한 옵션을 고려하며 여러 선택지 중에서 결정을 내리는 것을 좋아합니다.
유저의 목표:유저는 체리 향이 나고 너무 세지 않은 와인을 찾고 구매하려고 합니다. 유저의 예산은 7만원 이하이며, 여러 가지 제안을 바탕으로 정보에 입각한 결정을 내릴 수 있도록 원합니다.
대화:
유저: 체리향 나는 와인 찾고 있는데 추천 좀 해줄 수 있을까요?
어시스턴트: 네, 화이트 와인과 레드 와인 중 어떤 것을 선호하시나요?
유저: 흠, 그런거까지는 잘 몰라요. 
어시스턴트: 그럼 와인의 단맛에 대한 선호도를 알려주실 수 있을까요? 달콤하거나 건조한 것 중 어느 것을 선호하시나요?
유저: 아무래도 달달한 걸 좋아하는 편인 거 같아요.
어시스턴트: 마지막으로 예산은 어느 정도 생각하고 계신가요?
유저: 7만원 이하로 생각하고 있어요.
어시스턴트: 그럼 '투 티크 대모자' 와인을 추천드립니다. 이 와인은 세미 스위트화이트 와인으로 체리 향이 나며 가격이 6만원대입니다. 어떻게 생각하시나요?
유저: 오 좋아. 아무래도 안정적인 선택이 필요해서 다른 와인도 좀 더 추천해 주시겠어요?
어시스턴트: 네, '빌라 마리아 셀라 스타피노 그리지오'와 '조르디에르 블랑 드 블랑'도 추천드릴 수 있습니다. 두

In [57]:
result

"유저 페르소나: 유저는 완숙한 버섯과 블랙베리의 향이 나는 와인을 특히 좋아하며, 복잡한 풍미의 와인을 선호합니다. 알코올 도수는 14% 이하의 와인을 선호하여, 한 모금 떠 마실 때 부들거림을 느끼는 것을 좋아하지 않습니다. 유저의 예산은 10만원에서 20만원 사이이며 특별한 날에 마실 수 있는 좋은 와인을 찾는 중입니다. \n\n유저의 목표: 유저는 복잡한 풍미에 완숙한 버섯과 블랙베리의 향이 나고 알코올 도수가 14% 이하인 레드 와인을 구매하려고 합니다. 그의 예산은 10만원에서 20만원 사이입니다. \n\n대화: \n유저: 완숙한 버섯과 블랙베리 향이 나는 레드 와인 추천해줘\n어시스턴트: 복잡한 풍미를 조금 더 선호하시는 편인가요, 아니면 조금 더 단순한 풍미를 선호하시나요?\n유저: 복잡한 풍미를 선호해 \n어시스턴트: 알겠습니다. 그럼 와인의 알코올 도수는 얼마나 되는 것을 선호하시나요?\n유저: 14% 이하로 약간 부드러운 느낌의 와인을 좋아해\n어시스턴트: 좋아요, 그럼 추천 가격대는 어느 정도 생각하고 계세요? \n유저: 10만원에서 20만원 사이면 좋겠어\n어시스턴트: '베르테로 조르조 2016' 레드 와인을 추천드립니다. 이 와인은 완숙한 버섯와 블랙베리 향이 나면서도 복잡한 풍미를 가지고 있습니다. 알코올 도수는 13%로 조소합니다. 가격은 약 15만원으로 예산 범위에도 맞습니다. \n유저: 추가로 다른 와인도 추천해줄 수 있을까?\n어시스턴트: 물론입니다. 추가로 '더 레드 2015'와 '오가디로쏘 체테 2015'도 추천드립니다. 두 와인 모두 완숙한 버섯과 블랙베리의 향을 가지고 있으며, 복잡한 풍미를 느낄 수 있습니다. 알코올 도수는 14% 이하이며 가격은 각각 12만원과 18만원입니다. 이 와인들에 대해 어떻게 생각하시나요?"

아래의 generate_dialog_num에 생성을 원하는 대화의 수를 입력하면 for loop를 돌며 대화를 생성하고 이를 [생성된 대화 파일](./templates/examples/dialog_generate_prompt_examples.yaml)에 저장합니다.

In [91]:
yaml.indent(mapping=4, sequence=4)
generate_dialog_num = 1

for i in range(generate_dialog_num):
    example_selector = CustomExampleSelector('./templates/examples/dialog_generate_prompt_examples.yaml')
    
    prompt = FewShotPromptTemplate(
        prefix=prefix_prompt_template,
        example_selector=example_selector, 
        example_prompt=example_prompt,
        suffix=suffix_prompt_template, 
        input_variables=[]
    )
    dialog_generate_chain = LLMChain(
        llm=llm,
        prompt=prompt, 
        verbose=True, # 과정을 출력할지
        )
    result = dialog_generate_chain.run({})
    result = parser.parse(result)
    if result['conversation'].startswith('\n'):
        result['conversation'] = result['conversation'][1:]

    with open('./templates/examples/dialog_generate_prompt_examples.yaml', 'a', encoding='utf8') as outfile:
        yaml.dump([result], outfile)



> Entering new  chain...
Prompt after formatting:

여러분의 역할은 와인을 추천하는 어시스턴트와 유저 사이의 대화를 생성하는 것입니다.

아래 단계에 따라 문제를 해결하세요.
1단계. 유저의 페르소나를 생성합니다.
2단계. 유저가 달성하고자 하는 목표를 생성합니다.
3단계. 유저의 페르소나와 유저의 목표를 기반으로 대화를 생성합니다.

아래는 예시입니다.


유저 페르소나:유저는 다양한 종류의 와인을 즐깁니다. 새로운 종류의 와인을 탐색하는 것을 좋아하며, 러스틱한 풍미와 중간 정도의 바디감을 가진 와인을 선호합니다. 유저의 예산은 7만원 정도이며, 와인을 선택할 때 와인의 풍미, 식품과의 궁합, 가격 등을 고려합니다.

유저의 목표:유저는 이번 주말 친구와 함께 마실 새로운 와인을 찾고 있습니다. 사용자의 예산은 7만원 이하이며, 와인에 관한 많은 정보를 제공받아 와인을 선택하고자 합니다.

대화:
유저: 이번 주말 친구들과 함께 마실 새로운 와인을 추천해 주실 수 있을까요?
어시스턴트: 네, 알겠습니다. 어떤 풍미의 와인을 선호하시나요?
유저: 러스틱한 풍미와 중간 정도의 바디감을 가진 와인을 좋아해요.
어시스턴트: 그렇군요. 와인의 가격대는 어느 정도를 생각하시나요?
유저: 7만원 내외로 생각하고 있습니다. 
어시스턴트: 알겠습니다. 그러면 '투나파 패스 쇼라즈'라는 와인을 추천해드리겠습니다. 이는 호주에서 생산된 와인으로, 러스틱한 풍미와 중간정도의 바디감을 가지고 있습니다. 가격은 7만원 미만으로 합리적입니다. 어떠신가요?
유저: 좋네요, 그 외에 다른 와인도 추천해 줄 수 있을까요?
어시스턴트: 물론입니다. '엠메사지 페소니 아리히'라는 칠레 와인도 추천드립니다. 이 와인은 러스틱한 풍미와 중간 정도의 바디감을 지니고 있으며, 가격은 7만원 미만입니다. 이 중에서 마음에 드는 와인이 있으신가요?


이제 정해진 순서대로 유저와 어시스턴트 간의 대화를 생성합니다.


> Finished chain.


### 유저와 